# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [22]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [23]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [24]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [25]:
# Load the historical closing prices for bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [26]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [27]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [28]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [31]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 1

# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [33]:
# Use 70% of the data for training and the remaineder for testing
# YOUR CODE HERE!
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   test_size=0.3 
                                                   )

X_train.shape 


(378, 1)

In [36]:
# Use MinMaxScaler to scale the data between 0 and 1. 
# YOUR CODE HERE!
from sklearn.preprocessing import MinMaxScaler
sclr = MinMaxScaler()

sclr.fit(X)
X_train = sclr.transform(X_train)
X_test = sclr.transform(X_test)

sclr.fit(y)
y_train = sclr.transform(y_train)
y_test = sclr.transform(y_test)


In [37]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

#scaler.fit(np.array(y_vader_prob).reshape(0,1))
#y_vader_prob_norm = scaler.transform(np.array(y_vader_prob).reshape(0,1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [41]:
# Build the LSTM model. 
model = Sequential()
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# YOUR CODE HERE!
# Layer 1
model.add(LSTM(units=30, return_sequences=True, input_shape=(X_train.shape[1],1,)))
model.add(Dropout(0.2))
#layer2
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
# Output layer
model.add(LSTM(units=30))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [43]:
# Compile the model
# YOUR CODE HERE!

model.compile(optimizer="adam", loss = "mean_squared_error")


In [15]:
# Summarize the model
# YOUR CODE HERE!
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 30)             3840      
_________________________________________________________________
dropout (Dropout)            (None, 1, 30)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 30)             7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 30)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 31        
Total para

In [16]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
batch_size = 100
epochs = 10
model.fit(
    X_train_rnn,
    y_train_rnn,
    validation_data=(X_val_rnn, y_val_rnn),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
377/377 [==============================] - 9s 23ms/sample - loss: 0.0512
Epoch 2/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0269
Epoch 3/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0177
Epoch 4/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0113
Epoch 5/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0081
Epoch 6/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0051
Epoch 7/10
377/377 [==============================] - 4s 11ms/sample - loss: 0.0046
Epoch 8/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0039
Epoch 9/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0036
Epoch 10/10
377/377 [==============================] - 4s 10ms/sample - loss: 0.0035


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [17]:
# Evaluate the model
# YOUR CODE HERE!

163/163 [==============================] - 1s 5ms/sample - loss: 0.0236


0.023644124669097134

In [18]:
# Make some predictions
# YOUR CODE HERE!
y_rnn_pred = model.predict_classes(X_test_rnn, batch_size=100)

In [19]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [20]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,3670.919922,3625.541992
1,3670.919922,3663.253174
2,3912.570068,3663.253174
3,3924.239990,3833.080322
4,3974.050049,3841.282715


In [21]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
stocks.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)